In [12]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone
import time
from data_loader_s3 import upload_file

In [2]:
# Measure exec time
start = datetime.now()

In [3]:
### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

In [4]:
from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

### The pct calculations are not properly saved on the dataset unless I upload one previously saved. Why?
# The pipeline defined solve the issue by saving the calculated data on a new dataset but the calculations
# are done in the full dataset everytime the code runs.
# This practice is not scalable

# The pct could be calculated on the web interface for a given date selected by the users but this implies 
# more processing time

## Date of reference

In [5]:
df = df_market.groupby(by=["date","currency_code"]).mean()
df.reset_index(inplace=True)

### Dataset with implicit exchange of reference by currency:
### The exchange of reference corresponds to the first day of 2022 recorded on the dataset
date_reference = "2022-01-02"

exchange = exchange_reference(df, date_reference)
exchange = pd.DataFrame.from_dict(exchange, orient="index")

exchange.head()

/home/alber/anaconda3/envs/currency_tracker/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,0
ADA,NaN
AED,3.844643
AOA,NaN
ARS,196.947751
AUD,1.462145


In [6]:
df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

### Drop Null values:

df.dropna(subset=['pct'], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7021 entries, 1 to 8976
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               7021 non-null   datetime64[ns]
 1   currency_code      7021 non-null   object        
 2   time_stamp         7021 non-null   float64       
 3   volume_btc         7021 non-null   float64       
 4   avg_24h            7021 non-null   float64       
 5   avg_12h            6582 non-null   float64       
 6   avg_6h             5855 non-null   float64       
 7   avg_1h             3669 non-null   float64       
 8   altcoin            7021 non-null   float64       
 9   avg_24h_usd        7021 non-null   float64       
 10  implicit_exchange  7021 non-null   float64       
 11  pct                7021 non-null   float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 713.1+ KB


### Store csv and parquet files

In [8]:
# Saving historical data by appending:
df.to_csv(f'./data/analytics/csv/pct_change.csv',index = False, header = None, mode = 'a')

# Read historical data and transform into parquet :
df_results = pd.read_csv('./data/analytics/csv/pct_change.csv')
# Overwrite to update file with appended data
df_results.to_parquet(f'./data/analytics/parquet/pct_change.parquet',index=False)

### Saving on DB Version 2

In [9]:
engine = create_engine('sqlite:///data/CurrenciesDataBase_pct_change.db')
print(type(engine))
df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


In [10]:
end = dt.datetime.now()
print("Last Execution: ")
print(str(end))
print('Completed')

Last Execution: 
2022-07-12 13:53:18.611082
Completed


In [11]:
print("Execution time:", end-start)

Execution time: 0:00:02.926279


## Upload to S3 analytics/

In [13]:
file_name = f'./data/analytics/parquet/pct_change.parquet'
object_name = f"analytics/pct_change.parquet"
upload_file(file_name, "currencytracker", object_name)

'Data succesfully loaded to S3'